In [1]:
# this sets up basic packages
import numpy as np
import pandas as pd
import astropy.units as u
import astropy.cosmology.units as cu
import h5py

# this sets up matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# this sets up astropy
from astropy.io import fits
from astropy.wcs import WCS
from astropy.wcs.utils import pixel_to_skycoord
from astropy.utils.data import get_pkg_data_filename
from astropy.coordinates import SkyCoord, Angle, match_coordinates_sky, Distance
from astropy.table import Table, unique, setdiff

from regions import Regions, CircleSkyRegion

In [2]:
dream = fits.open('/Users/ngbinh/Downloads/DREaM_main.fits')
dream_data = Table(dream[1].data)

In [3]:
# this selects all the galaxies with 2 < z < 5, then gets their Mstar and SFR
relevant = dream_data['ID', 'RA', 'Dec', 'redshift', 'M_gal', 'logpsi'][dream_data['redshift']<5]
relevant = relevant[relevant['redshift']>2].to_pandas()

In [4]:
# this selects all the ones that are quenched (i.e. sSFR < -9.8)
quenched_ones = relevant[np.log10(10**relevant['logpsi'] / relevant['M_gal']) < -9.8]
quenched_ones = quenched_ones[quenched_ones['M_gal'] > 10**10.5]

In [5]:
relevant

,ID,RA,Dec,redshift,M_gal,logpsi
0,13241306,0.489857,0.010564,2.014664,1.273912e+07,-1.369086
1,13241307,0.489621,0.015140,2.014622,1.961021e+05,-2.058861
2,13241308,0.490752,-0.043852,2.014522,7.534733e+05,-2.632520
3,13241309,0.490896,-0.043992,2.014489,2.077482e+05,1.622524
4,13241310,0.492010,-0.043814,2.014393,1.295432e+05,-3.098835
...,...,...,...,...,...,...
12343026,25951780,-0.395015,0.470712,4.977378,1.483325e+05,1.476693
12343027,25951781,-0.325127,0.410665,4.984012,3.898026e+05,-1.798956
12343028,25951782,-0.052119,-0.499995,4.966891,2.134280e+05,-2.114437
12343029,25951783,0.170450,-0.499948,4.950807,2.765631e+06,-1.781149


There are too many quenched galaxies in this catalog, so let's just randomly sample 100 of them.

In [7]:
# this selects all the ones that are NOT quenched in relevant. the potential neighbors will be in here.
others = pd.concat([relevant, quenched_ones]).drop_duplicates(keep=False)

In [8]:
others

,ID,RA,Dec,redshift,M_gal,logpsi
0,13241306,0.489857,0.010564,2.014664,1.273912e+07,-1.369086
1,13241307,0.489621,0.015140,2.014622,1.961021e+05,-2.058861
2,13241308,0.490752,-0.043852,2.014522,7.534733e+05,-2.632520
3,13241309,0.490896,-0.043992,2.014489,2.077482e+05,1.622524
4,13241310,0.492010,-0.043814,2.014393,1.295432e+05,-3.098835
...,...,...,...,...,...,...
12343026,25951780,-0.395015,0.470712,4.977378,1.483325e+05,1.476693
12343027,25951781,-0.325127,0.410665,4.984012,3.898026e+05,-1.798956
12343028,25951782,-0.052119,-0.499995,4.966891,2.134280e+05,-2.114437
12343029,25951783,0.170450,-0.499948,4.950807,2.765631e+06,-1.781149


In [9]:
quenched_sample = quenched_ones.sample(n=100)

In [10]:
# this creates SkyCoord objects for the quenched sample and the non-quenched ones
quenched_cat = SkyCoord(ra=quenched_sample['RA'].values*u.degree, dec=quenched_sample['Dec'].values*u.degree)
others_cat = SkyCoord(ra=others['RA'].values*u.degree, dec=others['Dec'].values*u.degree)

In [11]:
all_neighbor_ids = []

# this looks for neighbors for each of the quenched ones within 15 arcsec
for quenched_object in quenched_cat:
    
    # use separation to find nearby objects ("nearby" in terms of RA and Dec) in the catalog
    d2d = quenched_object.separation(others_cat)
    catalogmsk = d2d < 15*u.arcsec
    neighbor_ids = others['ID'].values[catalogmsk]

    all_neighbor_ids.append(neighbor_ids)

In [12]:
all_neighbor_ids = np.array(all_neighbor_ids, dtype='object')

In [13]:
quenched_sample

,ID,RA,Dec,redshift,M_gal,logpsi
1310597,14698440,0.447830,-0.381314,2.250043,5.659243e+10,-0.267845
1160003,14547846,-0.339924,0.471336,2.178209,1.291916e+11,-0.065768
2221508,15609351,0.106993,-0.381009,2.405133,7.820793e+10,-0.172671
7284966,20672809,-0.249836,0.308019,3.375109,1.087812e+11,0.107885
1257213,14645056,-0.392758,-0.323628,2.128645,5.960322e+10,-0.430760
...,...,...,...,...,...,...
673702,14061545,0.391681,-0.214795,2.158754,1.798941e+11,0.185749
2048704,15436547,-0.446243,-0.059027,2.257439,4.051551e+10,-0.457609
4961549,18349392,0.445695,0.456719,2.799548,6.562267e+10,-0.176864
3038736,16426579,0.068834,0.433229,2.499460,4.376592e+10,-0.368109


# CRITERIA: |z$_{QG}$ - z$_{gal}$| < 0.5 and M$_{star}$ >= threshold from the neighbors of the 9 QGs

In [15]:
mega_frame = []

for idx in range(quenched_sample.shape[0]):
    
    # this retrieves the info of the galaxy in the quenched sample
    quenched_info = quenched_sample.iloc[idx]
#    z_QG
#    z_QG = quenched_sample['redshift'].values[idx]

    # this retrieves the IDs of the neighbor candidates around the QG
    neighbor_id_array = all_neighbor_ids[idx]

    # this locates the galaxies with the IDs above, then retrieves their photometric redshifts and IDs.
    # note: the IDs are retrieved in order to keep track of which galaxies we're talking about.
    neighbor_zphot_df = others.loc[others['ID'].isin(neighbor_id_array)][['ID', 'RA', 'Dec', 'redshift', 'M_gal']]
    
    # first, only select the neighbors whose stellar mass is above the mass threshold of the neighbors in our 9 QG sample.
    # we determined that mass threshold to be 1.9 * 10^6 Msun. however, because the observational survey might have
    # a different mass completeness limit, we just use a second threshold 10^9 Msun just to be safe.
    # first_passes = neighbor_zphot_df[neighbor_zphot_df['M_gal'] >= 1.9*10**6]
    first_passes = neighbor_zphot_df[neighbor_zphot_df['M_gal'] >= 10**9]

    # second, only select the neighbors whose redshift is within 0.5 from the redshift of the simulated QG.
    second_passes = first_passes[np.abs(quenched_info['redshift'] - first_passes['redshift'].values) <= 0.5]

    # this adds the info of the host QG 
    new_data = {"host_ID": [int(quenched_info['ID'])]*second_passes.shape[0],
              "host_RA": [quenched_info['RA']]*second_passes.shape[0],
              "host_Dec": [quenched_info['Dec']]*second_passes.shape[0],
               "host_redshift": [quenched_info['redshift']]*second_passes.shape[0]}
    second_passes = second_passes.assign(**new_data)

    mega_frame.append(second_passes)

In [16]:
mega_df = pd.concat(mega_frame)

In [17]:
mega_df.to_csv('DREaM_quenched_and_nbrs.csv')

In [18]:
max(np.abs(mega_df['host_redshift'].values - mega_df['redshift'].values))

0.49502884281672044